In [1]:
import os
import sys
import glob
import imageio
import numpy as np
import scipy.io as sio
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec

In [2]:
plt.rcParams['figure.figsize'] = [8.0, 4.0]
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

plt.rcParams['font.size'] = 10
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams['lines.linewidth'] = 1

In [3]:
class resonance_lines(object):
	
	def __init__(self, Qx_range, Qy_range, orders, periodicity):
		
		if np.std(Qx_range):
			self.Qx_min = np.min(Qx_range)
			self.Qx_max = np.max(Qx_range)
		else:
			self.Qx_min = np.floor(Qx_range)-0.05
			self.Qx_max = np.floor(Qx_range)+1.05
		if np.std(Qy_range):
			self.Qy_min = np.min(Qy_range)
			self.Qy_max = np.max(Qy_range)
		else:
			self.Qy_min = np.floor(Qy_range)-0.05
			self.Qy_max = np.floor(Qy_range)+1.05

		self.periodicity = periodicity
									
		nx, ny = [], []

		for order in np.nditer(np.array(orders)):
			t = np.array(range(-order, order+1))
			nx.extend(order - np.abs(t))
			ny.extend(t)
		nx = np.array(nx)
		ny = np.array(ny)
	
		cextr = np.array([nx*np.floor(self.Qx_min)+ny*np.floor(self.Qy_min), \
						  nx*np.ceil(self.Qx_max)+ny*np.floor(self.Qy_min), \
						  nx*np.floor(self.Qx_min)+ny*np.ceil(self.Qy_max), \
						  nx*np.ceil(self.Qx_max)+ny*np.ceil(self.Qy_max)], dtype='int')
		cmin = np.min(cextr, axis=0)
		cmax = np.max(cextr, axis=0)
		res_sum = [range(cmin[i], cmax[i]+1) for i in xrange(cextr.shape[1])]								
		self.resonance_list = zip(nx, ny, res_sum)
		
	def plot_resonance(self, figure_object = None):	
		plt.ion()
		if figure_object:
			fig = figure_object
			plt.figure(fig.number)
		else:
			fig = plt.figure()
		Qx_min = self.Qx_min
		Qx_max = self.Qx_max
		Qy_min = self.Qy_min
		Qy_max = self.Qy_max 
		plt.xlim(Qx_min, Qx_max)
		plt.ylim(Qy_min, Qy_max)
		plt.xlabel('Qx')
		plt.ylabel('Qy')
		for resonance in self.resonance_list:
			nx = resonance[0]
			ny = resonance[1]
			for res_sum in resonance[2]:
				if ny:
					line, = plt.plot([Qx_min, Qx_max], \
					    [(res_sum-nx*Qx_min)/ny, (res_sum-nx*Qx_max)/ny])
				else:
					line, = plt.plot([np.float(res_sum)/nx, np.float(res_sum)/nx],[Qy_min, Qy_max])
				if ny%2:
					plt.setp(line, linestyle='--') # for skew resonances
				if res_sum%self.periodicity:
					plt.setp(line, color='b')	# non-systematic resonances
				else:
					plt.setp(line, color='r', linewidth=2.0) # systematic resonances
		plt.draw()
		return fig
		
	def print_resonances(self):
		for resonance in self.resonance_list:
			for res_sum in resonance[2]:
				'''
				print str(resonance[0]).rjust(3), 'Qx ', ("+", "-")[resonance[1]<0], \
					  str(abs(resonance[1])).rjust(2), 'Qy = ', str(res_sum).rjust(3), \
					  '\t', ("(non-systematic)", "(systematic)")[res_sum%self.periodicity==0]
				'''
				print '%s %s%s = %s\t%s'%(str(resonance[0]).rjust(2), ("+", "-")[resonance[1]<0], \
						str(abs(resonance[1])).rjust(2), str(res_sum).rjust(4), \
						("(non-systematic)", "(systematic)")[res_sum%self.periodicity==0])


In [15]:
# List of folders to iterate through
sims = []
V = True
H = False

if V:
    sims.append('03_SbS_Tomo_Lattice_V_24/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_23/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_22/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_21/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_20/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_19/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_18/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_17/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_16/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_15/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_14/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_13/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_12/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_11/bunch_output/')
    sims.append('03_SbS_Tomo_Lattice_V_10/bunch_output/')
if H:
    sims.append('04_SbS_Tomo_Lattice_H_07/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_08/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_09/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_10/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_11/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_12/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_13/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_14/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_15/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_16/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_17/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_18/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_19/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_20/bunch_output/')
    sims.append('04_SbS_Tomo_Lattice_H_21/bunch_output/')

for sim in sims:
    print sim

03_SbS_Tomo_Lattice_V_24/bunch_output/
03_SbS_Tomo_Lattice_V_23/bunch_output/
03_SbS_Tomo_Lattice_V_22/bunch_output/
03_SbS_Tomo_Lattice_V_21/bunch_output/
03_SbS_Tomo_Lattice_V_20/bunch_output/
03_SbS_Tomo_Lattice_V_19/bunch_output/
03_SbS_Tomo_Lattice_V_18/bunch_output/
03_SbS_Tomo_Lattice_V_17/bunch_output/
03_SbS_Tomo_Lattice_V_16/bunch_output/
03_SbS_Tomo_Lattice_V_15/bunch_output/
03_SbS_Tomo_Lattice_V_14/bunch_output/
03_SbS_Tomo_Lattice_V_13/bunch_output/
03_SbS_Tomo_Lattice_V_12/bunch_output/
03_SbS_Tomo_Lattice_V_11/bunch_output/
03_SbS_Tomo_Lattice_V_10/bunch_output/


# Iterate over sims, read in bunch outputs, plot footprints with histograms

In [ ]:
gif = False
for sim in sims:    
    print 'Entering folder:', sim
    # Read all file names
    plt.close('all')
    filename = 'mainbunch'
    files = glob.glob(sim + filename + '*.mat')
    files.sort()

    # Remove bunch before tracking - no tunespread
    for i, file in enumerate(files):
        if 'mainbunch_-000001.mat' in file:
            files.remove(file)
            
    gifnames =[]

    # Iterate over bunch files and plot footprints
    for i, file in enumerate(files):

        folder = file.split('/')[0] + '/' + file.split('/')[1]
        tune = file.split('/')[0][-2:]
        case = file.split('/')[0][-4]
        tune_tit = '6.'+tune
        tune_sav = '6p'+tune 

        try: 
            turn = int(file.split('mainbunch_')[-1][:-4])
            turn = '%04d'%turn
        except:
            turn = ''

        print '\t Plotting ', case, ' scan tune =', tune_tit, ' turn = ', turn, ' with histograms'

        particles = sio.loadmat(file, squeeze_me=True,  struct_as_record=False)['particles']
        x  = particles.x
        xp = particles.xp
        y  = particles.y
        yp = particles.yp
        z  = particles.z
        dE = particles.dE

        fontsize=15

        qx = particles.ParticlePhaseAttributes[2,:]
        qy = particles.ParticlePhaseAttributes[3,:]
        qx[np.where(qx>0.5)] -= 1
        qy[np.where((qy>0.6) & (qx<0.25))] -= 1

        resonances = resonance_lines((5.75, 6.25),(5.75, 6.25),(1,2,3,4),10)
        fontsize=17

        f, ax = plt.subplots(1, figsize=(6,6))
        gridspec.GridSpec(3,3)
        #f.subplots_adjust(hspace = 0)	# Horizontal spacing between subplots
        f.subplots_adjust(wspace = 0)	# Vertical spacing between subplots


        my_cmap = plt.cm.jet
        my_cmap.set_under('w',1)

        r = resonances

        title = str( case + ' ' + tune_tit + ' turn ' + turn  )    

        # First subplot
        plt.subplot2grid((3,3), (0,0), colspan=2, rowspan=1)   
        plt.hist(6+qx, bins=1000, range=(r.Qx_min, r.Qx_max)) #, norm=mcolors.PowerNorm(gamma))
        plt.ylabel('Frequency')
        plt.grid(which='both')
        plt.title(title, fontsize=fontsize)

        # Main plot
        plt.subplot2grid((3,3), (1,0), colspan=2, rowspan=2)
        plt.hist2d(6+qx, 6+qy, bins=1000, cmap=my_cmap, vmin=1, range=[[r.Qx_min, r.Qx_max], [r.Qy_min, r.Qy_max]]) #, norm=mcolors.PowerNorm(gamma))
        plt.xlabel(r'Q$_x$')
        plt.ylabel(r'Q$_y$')
        resonances.plot_resonance(f)

        # Second subplot
        plt.subplot2grid((3,3), (1,2), colspan=1, rowspan=2)    
        plt.hist(6+qy, bins=1000, range=(r.Qy_min, r.Qy_max), orientation=u'horizontal') #, norm=mcolors.PowerNorm(gamma))
        plt.xlabel('Frequency')
        plt.grid(which='both')

        current_axis = plt.gca()
        #current_axis.axes.get_yaxis().set_visible(False)

        ax.xaxis.label.set_size(fontsize)
        ax.yaxis.label.set_size(fontsize)

        ax.tick_params(labelsize=fontsize)

        plt.tight_layout()
        savename = str(folder + '/Tune_Footprint_' + case + '_' + tune_sav + '_turn_' + turn + '_hist.png' )
        gifnames.append(savename)
        f.savefig(savename, dpi=500)
        plt.close(f)
        
    if gif:
        images = []
        for filename in gifnames:
            images.append(imageio.imread(filename))    
            folder = filename.split('/')[0] + '/' + filename.split('/')[1] + '/'
            savename = str( folder + 'Tune_Footprints.gif')
        imageio.mimsave(savename, images)


Entering folder: 03_SbS_Tomo_Lattice_V_24/bunch_output/
	 Plotting  V  scan tune = 6.24  turn =  0001  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0002  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0003  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0004  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0005  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0006  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0007  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0008  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0010  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0020  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0030  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0040  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0050  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0060  with histograms
	 Plotting  V  scan tune = 6.24  turn =  0070  with histograms

	 Plotting  V  scan tune = 6.20  turn =  0007  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0008  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0010  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0020  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0030  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0040  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0050  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0060  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0070  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0080  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0090  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0199  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0399  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0599  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0799  with histograms
	 Plotting  V  scan tune = 6.20  turn =  0874  with his

	 Plotting  V  scan tune = 6.16  turn =  0060  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0070  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0080  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0090  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0199  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0399  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0599  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0799  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0874  with histograms
	 Plotting  V  scan tune = 6.16  turn =  0999  with histograms
	 Plotting  V  scan tune = 6.16  turn =  1199  with histograms
	 Plotting  V  scan tune = 6.16  turn =  1399  with histograms


# Individual plotting (single sim)

In [4]:
plt.close('all')
#source_dir =  '04_SbS_Tomo_Lattice_H_07/bunch_output/'
source_dir =  '03_SbS_Tomo_Lattice_V_10/bunch_output/'

filename = 'mainbunch'
files = glob.glob(source_dir + filename + '*.mat')
files.sort()

# Remove bunch before tracking - no tunespread
for i, file in enumerate(files):
    if 'mainbunch_-000001.mat' in file:
        files.remove(file)
files

['03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000001.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000002.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000003.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000004.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000005.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000006.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000007.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000008.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000010.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000020.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000030.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000040.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000050.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000060.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_000070.mat',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/mainbunch_00008

# Simple tune footprint

In [ ]:
for i, file in enumerate(files):
    
    folder = file.split('/')[0] + '/' + file.split('/')[1]
    tune = file.split('/')[0][-2:]
    case = file.split('/')[0][-4]
    tune_tit = '6.'+tune
    tune_sav = '6p'+tune 
    
    try: 
        turn = int(file.split('mainbunch_')[-1][:-4])
        turn = '%04d'%turn
    except:
        turn = ''
        
    print '\t Plotting ', case, ' scan tune =', tune_tit, ' turn = ', turn
        
    particles = sio.loadmat(file, squeeze_me=True,  struct_as_record=False)['particles']
    x  = particles.x
    xp = particles.xp
    y  = particles.y
    yp = particles.yp
    z  = particles.z
    dE = particles.dE

    fontsize=15

    qx = particles.ParticlePhaseAttributes[2,:]
    qy = particles.ParticlePhaseAttributes[3,:]
    qx[np.where(qx>0.5)] -= 1
    qy[np.where((qy>0.6) & (qx<0.25))] -= 1

    resonances = resonance_lines((5.75, 6.25),(5.75, 6.25),(1,2,3,4),10)
    fontsize=17
    
    f, ax = plt.subplots(1,figsize=(6,6))
    
    my_cmap = plt.cm.jet
    my_cmap.set_under('w',1)
    
    r = resonances
    
    ax.hist2d(6+qx, 6+qy, bins=1000, cmap=my_cmap, vmin=1, range=[[r.Qx_min, r.Qx_max], [r.Qy_min, r.Qy_max]]) #, norm=mcolors.PowerNorm(gamma))
    
    resonances.plot_resonance(f)
    
    ax.xaxis.label.set_size(fontsize)
    ax.yaxis.label.set_size(fontsize)
    
    ax.tick_params(labelsize=fontsize)
    
    title = str( case + ' ' + tune_tit + ' turn ' + turn  )    
    plt.title(title, fontsize=fontsize)
    plt.tight_layout()
    savename = str(folder + '/Tune_Footprint_' + case + '_' + tune_sav + '_turn_' + turn + '.png' )
    f.savefig(savename, dpi=500)
    plt.close(f)

# Add x and y histograms

In [7]:
gifnames =[]

for i, file in enumerate(files):
    
    folder = file.split('/')[0] + '/' + file.split('/')[1]
    tune = file.split('/')[0][-2:]
    case = file.split('/')[0][-4]
    tune_tit = '6.'+tune
    tune_sav = '6p'+tune 
    
    try: 
        turn = int(file.split('mainbunch_')[-1][:-4])
        turn = '%04d'%turn
    except:
        turn = ''
        
    print '\t Plotting ', case, ' scan tune =', tune_tit, ' turn = ', turn, ' with histograms'
        
    particles = sio.loadmat(file, squeeze_me=True,  struct_as_record=False)['particles']
    x  = particles.x
    xp = particles.xp
    y  = particles.y
    yp = particles.yp
    z  = particles.z
    dE = particles.dE

    fontsize=15

    qx = particles.ParticlePhaseAttributes[2,:]
    qy = particles.ParticlePhaseAttributes[3,:]
    qx[np.where(qx>0.5)] -= 1
    qy[np.where((qy>0.6) & (qx<0.25))] -= 1

    resonances = resonance_lines((5.75, 6.25),(5.75, 6.25),(1,2,3,4),10)
    fontsize=17
    
    f, ax = plt.subplots(1, figsize=(6,6))
    gridspec.GridSpec(3,3)
    #f.subplots_adjust(hspace = 0)	# Horizontal spacing between subplots
    f.subplots_adjust(wspace = 0)	# Vertical spacing between subplots
    
    
    my_cmap = plt.cm.jet
    my_cmap.set_under('w',1)
    
    r = resonances
    
    title = str( case + ' ' + tune_tit + ' turn ' + turn  )    
    
    # First subplot
    plt.subplot2grid((3,3), (0,0), colspan=2, rowspan=1)   
    plt.hist(6+qx, bins=1000, range=(r.Qx_min, r.Qx_max)) #, norm=mcolors.PowerNorm(gamma))
    plt.ylabel('Frequency')
    plt.grid(which='both')
    plt.title(title, fontsize=fontsize)
    
    # Main plot
    plt.subplot2grid((3,3), (1,0), colspan=2, rowspan=2)
    plt.hist2d(6+qx, 6+qy, bins=1000, cmap=my_cmap, vmin=1, range=[[r.Qx_min, r.Qx_max], [r.Qy_min, r.Qy_max]]) #, norm=mcolors.PowerNorm(gamma))
    plt.xlabel(r'Q$_x$')
    plt.ylabel(r'Q$_y$')
    resonances.plot_resonance(f)
    
    # Second subplot
    plt.subplot2grid((3,3), (1,2), colspan=1, rowspan=2)    
    plt.hist(6+qy, bins=1000, range=(r.Qy_min, r.Qy_max), orientation=u'horizontal') #, norm=mcolors.PowerNorm(gamma))
    plt.xlabel('Frequency')
    plt.grid(which='both')
    
    current_axis = plt.gca()
    #current_axis.axes.get_yaxis().set_visible(False)
    
    ax.xaxis.label.set_size(fontsize)
    ax.yaxis.label.set_size(fontsize)
    
    ax.tick_params(labelsize=fontsize)
    
    plt.tight_layout()
    savename = str(folder + '/Tune_Footprint_' + case + '_' + tune_sav + '_turn_' + turn + '_hist.png' )
    gifnames.append(savename)
    f.savefig(savename, dpi=500)
    plt.close(f)


	 Plotting  V  scan tune = 6.10  turn =  0001

	 Plotting  V  scan tune = 6.10  turn =  0002

	 Plotting  V  scan tune = 6.10  turn =  0003

	 Plotting  V  scan tune = 6.10  turn =  0004

	 Plotting  V  scan tune = 6.10  turn =  0005

	 Plotting  V  scan tune = 6.10  turn =  0006

	 Plotting  V  scan tune = 6.10  turn =  0007

	 Plotting  V  scan tune = 6.10  turn =  0008

	 Plotting  V  scan tune = 6.10  turn =  0010

	 Plotting  V  scan tune = 6.10  turn =  0020

	 Plotting  V  scan tune = 6.10  turn =  0030

	 Plotting  V  scan tune = 6.10  turn =  0040

	 Plotting  V  scan tune = 6.10  turn =  0050

	 Plotting  V  scan tune = 6.10  turn =  0060

	 Plotting  V  scan tune = 6.10  turn =  0070

	 Plotting  V  scan tune = 6.10  turn =  0080

	 Plotting  V  scan tune = 6.10  turn =  0090

	 Plotting  V  scan tune = 6.10  turn =  0199

	 Plotting  V  scan tune = 6.10  turn =  0399

	 Plotting  V  scan tune = 6.10  turn =  0599

	 Plotting  V  scan tune = 6.10  turn =  0799

	 Plotting  

In [8]:
gifnames

['03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0001_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0002_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0003_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0004_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0005_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0006_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0007_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0008_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0010_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0020_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0030_hist.png',
 '03_SbS_Tomo_Lattice_V_10/bunch_output/Tune_Footprint_V_6p10_turn_0040_hist.png',
 '03

# Make a GIF

In [10]:
images = []
for filename in gifnames:
    images.append(imageio.imread(filename))    
    folder = filename.split('/')[0] + '/' + filename.split('/')[1] + '/'
    savename = str( folder + 'Tune_Footprints.gif')
imageio.mimsave(savename, images)